Image classification with CNNs
================

The goal of this exercise is to implement a specific CNN architecture with PyTorch and train it on the CIFAR-10 image classification dataset. We will start by introducing the dataset and then implement a `nn.Module` and a useful `Solver` class. Seperating the model from the actual training has proven itself as a sensible design decision. By the end of this exercise you should have succesfully trained your (possible) first CNN model and have a boilerplate `Solver` class which you can reuse for the next exercise and your future research projects.

For an inspiration on how to implement a model or the solver class you can have a look at [these](https://github.com/pytorch/examples) PyTorch examples.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import torch
from torch.autograd import Variable

#from dl4cv.classifiers.classification_cnn import ClassificationCNN
from yz.data_utils import get_Cancer_datasets
#torch.set_default_tensor_type('torch.FloatTensor')

csv_full_name = '/home/ubuntu/dl4cvproject/data/train.csv'
img_folder_full_name = '/home/ubuntu/dl4cvproject/data/train_256'

%matplotlib inline
plt.rcParams['figure.figsize'] = (10.0, 8.0) # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

# for auto-reloading external modules
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2

Cancer Dataset
=========

Since the focus of this exercise should be neural network models and how to successfully train them, we provide you with preprocessed and prepared datasets. For an even easier management of the train, validation and test data pipelines we provide you with custom `torch.utils.data.Dataset` classes. Use the official [documentation](http://pytorch.org/docs/data.html) to make yourself familiar with the `Dataset` and `DataLoader` classes. Think about how you have to integrate them in your training loop and have a look at the data preprocessing steps in `dl4cv/data_utils.py`.

The `num_workers` argument of the `DataLoader` class allows you to preprocess data with multiple threads.

<div class="alert alert-info">
    <h3>Note</h3>
    <p>In this case we generated the `Dataset` classes after we applied all the preprocessing steps. Other datasets or random data augmentation might require an online preprocessing which can be integrated into the `Dataset` classes. See `torchvision.Transform` for examples.</p>
</div>

In [2]:
# Load the (preprocessed) CIFAR10 data. The preprocessing includes
# channel swapping, normalization and train-val-test splitting.
# Loading the datasets might take a while.

train_data, val_data, test_data, train_label_list = get_Cancer_datasets(csv_full_name=csv_full_name,img_folder_full_name=img_folder_full_name)
#train_data, val_data, test_data, train_label_list = get_Cancer_datasets(csv_full_name=csv_full_name,img_folder_full_name=img_folder_full_name,
#                                                                                  num_training=50, num_validation=20, num_test=30)
# X = get_Cancer_datasets(csv_full_name=csv_full_name,img_folder_full_name=img_folder_full_name, num_training=800, num_validation=100, num_test=100)
print("Train size: %i" % len(train_data))
print("Val size: %i" % len(val_data))
print("Test size: %i" % len(test_data))

100%|█████████▉| 18545/18577 [00:42<00:00, 441.09it/s]

transforming...
X.shape:(18577, 256, 256, 3)
X_original.shape:(18577, 256, 256)
mean:125.83685163263934



100%|██████████| 18577/18577 [00:00<00:00, 1188021.61it/s]

std:63.15855322359238
Done transforming...
Getting labels
submasking...
num_training:16000
OK...
Train size: 16000
Val size: 1250
Test size: 1327


In [3]:
#print(len(train_data))
print(type(train_data), type(val_data), type(test_data))
print()
for i in range(1):
    inputs, labels = train_data[i]
    print('label_type: {}'.format(type(labels)))

img, lb = train_data[i]
print('img_type: {}'.format(type(img)))
print('img_shape:{}'.format(img.size()))

<class 'yz.data_utils.CancerData'> <class 'yz.data_utils.CancerData'> <class 'yz.data_utils.CancerData'>

label_type: <class 'numpy.int64'>
img_type: <class 'torch.FloatTensor'>
img_shape:torch.Size([3, 224, 224])


In [4]:
from tqdm import tqdm
label_set = set()
cnt = [0] * 14
for i in tqdm(range(len(train_data))):
    _, labels = train_data[i]
    for num in range(len(cnt)):
        if labels == num:70
            cnt[num] += 1
    label_set.add(labels)
print(label_set)
print(cnt)

IndentationError: unexpected indent (<ipython-input-4-a0eb1ccc0802>, line 8)

100%|█████████▉| 18545/18577 [01:00<00:00, 308.82it/s]

In [ ]:
# find a sample with certain index

class_label = 13
for i in range(len(train_data)):
    _, labels = train_data[i]
    if labels == class_label:
        print(i)
        break

Visualize Examples
------------------

To make yourself familiar with the dataset we visualize some examples. We show a few examples from each class. Note that we have to revert (transposition and mean subtraction) some preprocessing steps.

In [ ]:

index = 10009
img, _ = train_data[index]
print(torch.sum(img > 0))
print(img)
to_plot = img.cpu().numpy() #+ mean_image
print(to_plot.shape)
plt.imshow(to_plot[0])
plt.show()

## Model Architecture and Forward Pass 

After you understood the core concepts of PyTorch and have a rough idea on how to implement your own model, complete the initialization and forward methods of the `ClassificationCNN` in the `dl4cv/classifiers/classification_cnn.py` file. Note that we do not have to implement a backward pass since this is automatically handled by the `autograd` package.

Use the cell below to check your results:

## Training and Validation with the Solver
We train and validate our previously generated model with a seperate `Solver` class defined in `dl4cv/solver.py`. Complete the `.train()` method and try to come up with an efficient iteration scheme as well as an informative training logger.

Use the cells below to test your solver. A nice trick is to train your model with just a few training samples. You should be able to overfit small datasets, which will result in very high training accuracy and comparatively low validation accuracy.

<div class="alert alert-info">
    <h3>Note</h3>
    <p>As seen below, the design of our `Solver` class is indepdenent of the particular model or data pipeline. This facilitates the reuse of the class and its modular structure allows the training of different models.</p>
</div>

Plotting the loss, training accuracy, and validation accuracy should show clear overfitting:

## Train the Network
Now train your model with the full dataset. By training a `ThreeLayerCNN` model for one epoch, you should already achieve greater than 40% accuracy on the validation set. If your training is painfully slow check if you did not forget to call the `nn.Module.cuda()` method.

For the overfitting example we provided you with a set of hyperparamters (`hidden_dim`, `lr`, `weight_decay`, ...). You can start with the same parameter values but in order to maximize your accuracy you should try to train multiple models with different sets of hyperparamters. This process is called hyperparameter optimization.

In [ ]:
torch.cuda.is_available()

In [ ]:
from yz.classifiers.classification_cnn import ClassificationCNN
from yz.classifiers.transferred_alexnet import alexnet
from yz.solver import Solver
from yz.data_utils import get_balanced_weights
from torchvision import models
import torch.nn as nn

weights = get_balanced_weights(train_label_list, 14)
sampler = torch.utils.data.sampler.WeightedRandomSampler(weights, len(weights))
train_loader = torch.utils.data.DataLoader(train_data, batch_size=50, shuffle=False, sampler=sampler, num_workers=8)
val_loader = torch.utils.data.DataLoader(val_data, batch_size=50, shuffle=False, num_workers=8)


model = models.resnet101(pretrained=True)
in_features = model.fc.in_features
model.fc = nn.Linear(in_features, 14)

"""
VGG16

model = models.vgg16(pretrained=True)
model.classifier  = nn.Sequential(
            nn.Linear(512 * 7 * 7, 4096),
            nn.ReLU(True),
            nn.Dropout(),
            nn.Linear(4096, 4096),
            nn.ReLU(True),
            nn.Dropout(),
            nn.Linear(4096, 14),
)
"""

#list(model.classifier.children())[:-1] = nn.Linear(4096, 14)  
if torch.cuda.is_available():
    print('Cuda available')
    model.cuda()


solver = Solver()
solver.train(model, train_loader, val_loader, log_nth=1, num_epochs=1)

In [ ]:
train_loader = torch.utils.data.DataLoader(train_data, batch_size=50, shuffle=False, sampler=sampler, num_workers=8, drop_last=True)
val_loader = torch.utils.data.DataLoader(val_data, batch_size=50, shuffle=False, num_workers=8)
solver = Solver()
solver.train(model, train_loader, val_loader, log_nth=2, num_epochs=4)

# Test your Model
Run your best model on the test set. You should easily achieve a score above 10% (random guessing for a classification task with 10 classes) accuracy on the given test set:

In [ ]:
from tqdm import tqdm
test_loader = torch.utils.data.DataLoader(test_data, batch_size=30, shuffle=False, num_workers=8)

scores = []
for inputs, target in tqdm(test_loader):
    #print(type(target))
    inputs, targets = Variable(inputs), Variable(target)
    if torch.cuda.is_available:
        inputs, targets = inputs.cuda(), targets.cuda()

    outputs = model(inputs)
    _, preds = torch.max(outputs, 1)
    scores.extend((preds == targets).data.cpu().numpy())
    
print('Test set accuracy: %f' % np.mean(scores))

## Get final test data

In [ ]:
v = csv_test['detected'].values

In [ ]:
from yz.data_utils import get_Cancer_datasets
csv_full_name = '/home/ubuntu/dl4cvproject/data/test.csv'
img_folder_full_name = '/home/ubuntu/dl4cvproject/data/test_256'
test_X, csv_test = get_Cancer_datasets(csv_full_name=csv_full_name,img_folder_full_name=img_folder_full_name, mode='upload')

In [ ]:
index = 20
img = test_X[index]
print(torch.sum(img > 0))
print(img)
print(type(img))
to_plot = img.cpu().numpy() #+ mean_image
print(to_plot.shape)
plt.imshow(to_plot[0])
plt.show()

In [ ]:
print(type(csv_test))
print(len(test_X))

In [ ]:
try:
    del csv_test['age']
except KeyError as e:
    print(e)
try:
    del csv_test['gender']
except KeyError as e:
    print(e)
try:
    del csv_test['view_position']
except KeyError as e:
    print(e)
try:
    del csv_test['image_name']
except KeyError as e:
    print(e)
try:
    del csv_test['detected']
except KeyError as e:
    print(e)

print(list(csv_test))

In [ ]:
inputs = test_X[0]
print(inputs.unsqueeze(0).size())
test = torch.cat((inputs.unsqueeze(0), inputs.unsqueeze(0)), 0)
print(test.size())
if torch.cuda.is_available:
    inputs = inputs.cuda()
    test.cuda()
outputs = model(Variable(test).cuda())
_, preds = torch.max(outputs, 1)
print(preds)

## Prediction and Submission CSV

In [ ]:
try:
    del csv_test['age']
except KeyError as e:
    print(e)
try:
    del csv_test['gender']
except KeyError as e:
    print(e)
try:
    del csv_test['view_position']
except KeyError as e:
    print(e)
try:
    del csv_test['image_name']
except KeyError as e:
    print(e)
try:
    del csv_test['detected']
except KeyError as e:
    print(e)

print(list(csv_test))

import pandas as pd
index = 0
jump = 30
detected = []
pred_set = set()
debug_length = 10
for i in tqdm(range(len(test_X))):
#for i in tqdm(range(debug_length)):
    #start = index
    #end = index + jump
    #if end >= (test_X.size()[0]) :
    #    end = test_X.size()[0]
    tmp_pred_list = [0] * 14
    for trial in range(7):
        inputs = test_X[i]
        inputs = Variable(inputs.unsqueeze(0))
        if torch.cuda.is_available:
            inputs = inputs.cuda()
        outputs = model(inputs)
        _, preds = torch.max(outputs, 1)
        int_label = preds.data.cpu().numpy().tolist()[0]
        tmp_pred_list[int_label] += 1
    
    pred = tmp_pred_list.index(max(tmp_pred_list))
    #print(tmp_pred_list, pred)
    str_pred = 'class_' + str(pred + 1)
    detected.append(str_pred)
    ###
    #int_list_preds = preds.data.cpu().numpy().tolist()
    #for pred_num in int_list_preds:
    #    pred_set.add(pred_num + 1)
    #str_list_preds = [('class_' + str(pred_num + 1)) for pred_num in int_list_preds]
    #detected.extend(str_list_preds)
    ####
    #if end == test_X.size()[0]:
    #    break
    #index += jump

In [ ]:
print(pred_set)
csv_test['detected'] = pd.Series(detected)
csv_test.to_csv('submission.csv', index=False)
print(csv_test)

model.save("models/classification_cnn.model")